# Client Example

In [1]:
from langserve import RemoteRunnable

In [2]:
from typing import Optional, List
from pydantic import BaseModel, Field
import requests

class Person(BaseModel):
    age: Optional[int] = Field(None, description="The age of the person in years.")
    name: Optional[str] = Field(None, description="The name of the person.")
    alias: Optional[str] = Field(None, description="Alias, if any.")
    
class Root(BaseModel):
    """A list of people that are mentioend in the text."""
    people: List[Person] = Field(None, description="List of people")

In [7]:
Person.schema()

{'title': 'Person',
 'type': 'object',
 'properties': {'age': {'title': 'Age',
   'description': 'The age of the person in years.',
   'type': 'integer'},
  'name': {'title': 'Name',
   'description': 'The name of the person.',
   'type': 'string'},
  'alias': {'title': 'Alias',
   'description': 'Alias, if any.',
   'type': 'string'}}}

In [8]:
runnable = RemoteRunnable("http://localhost:8000/extract_text/")

In [9]:
text = """
My name is Chester. i am 42 years old. My friend Jane is a year older than me.
"""

In [10]:
Root.schema()

{'title': 'Root',
 'description': 'A list of people that are mentioend in the text.',
 'type': 'object',
 'properties': {'people': {'title': 'People',
   'description': 'List of people',
   'type': 'array',
   'items': {'$ref': '#/definitions/Person'}}},
 'definitions': {'Person': {'title': 'Person',
   'type': 'object',
   'properties': {'age': {'title': 'Age',
     'description': 'The age of the person in years.',
     'type': 'integer'},
    'name': {'title': 'Name',
     'description': 'The name of the person.',
     'type': 'string'},
    'alias': {'title': 'Alias',
     'description': 'Alias, if any.',
     'type': 'string'}}}}}

In [11]:
response = runnable.invoke({"text": text, "schema": Root.schema()})
response

{'extracted': {'people': [{'name': 'Chester', 'age': 42}, {'name': 'Jane'}]}}

In [12]:
instructions = "Redact all names using the characters `######`"
examples = [
    {
        "text": "My name is Grung. I am 100.",
        "output": Root(people=[Person(age=100, name="######")]).dict(),
    },
]

response = runnable.invoke(
    {
        "text": text,
        "schema": Root.schema(),
        "instructions": instructions,
        "examples": examples,
    }
)
response

{'extracted': {'people': [{'age': 42, 'name': '######', 'alias': 'Chester'},
   {'age': 43, 'name': '######', 'alias': 'Jane'}]}}